# Lista 3 - Finanças Quantitativas e Gerenciamento de Risco

- Vinicius Julião Ramos

## Questão 2

In [15]:
source("../utils/graficos.r")
source("../utils/funcoesAuxiliares.r");
source("../utils/model.r");
loadPackage("quantmod");
loadPackage("quadprog");
library("reshape2")
library("ggplot2")

options(scipen=999);
loadPackage("RCurl");
data <- getURL("http://www.dcc.ufmg.br/~arbex/portfolios/IBOV.csv", ssl.verifypeer=0L, followlocation=1L);
prices = read.csv(text = data, header = TRUE, sep = ",", stringsAsFactors = FALSE);
begin  = which(prices[,1] > 20190000)[1];
end    = which(prices[,1] > 20200000)[1];
os.prices = prices[end:nrow(prices), ];
is.prices = prices[begin:end, ];

### Passo 1:

Obter os retornos de insample por Markowitz

In [10]:
# removendo as colunas 'Date' e 'IBOV'
insample.prices = is.prices[,3:ncol(is.prices)];
outsample.prices = os.prices[,3:ncol(os.prices)];

# calculo da matriz de retornos simples
insample.returns = apply(insample.prices, 2, function(x){diff(x)/x[-length(x)]} );
outsample.returns = apply(outsample.prices, 2, function(x){diff(x)/x[-length(x)]} );

# calculo do vetor de retornos esperados (mu) e matriz de covariancias (Sigma)
insample.mu = matrix(colMeans(insample.returns), nrow=1)
colnames(insample.mu) = colnames(insample.returns);
insample.Sigma = cov(insample.returns);

N = ncol(insample.returns)
markowitz.D = insample.Sigma; # Q do modelo
markowitz.d = rep(0, N);      # c do modelo

markowitz.A = matrix(c(rep(1, N),
        insample.mu,
        diag(N),
        -diag(N)),
        ncol = N,
        byrow = TRUE);

markowitz.b = c(1, 0.0001, rep(-0.15, 2*N));

markowitz.meq  = 1;

markowitz.result <- solve.QP(Dmat = markowitz.D, dvec = markowitz.d, Amat=t(markowitz.A), bvec=markowitz.b,
                            meq=markowitz.meq);

markowitz.weights = markowitz.result$solution

names(markowitz.weights) <- colnames(insample.prices)
markowitz.mu = as.numeric(crossprod(markowitz.weights, as.vector(insample.mu)))
markowitz.sigma2 = (markowitz.result$val)*2
markowitz.sigma = sqrt(markowitz.sigma2)

markowitz.chart.data <- markowitz.weights[markowitz.weights>= 0.0001]

print(markowitz.weights)

      BOVA11        ABEV3        AZUL4        BTOW3        B3SA3        BBAS3 
-0.128297821  0.094513051 -0.049811930 -0.002387883 -0.135123154 -0.041617455 
       BBSE3        BRML3        BBDC3        BBDC4        BRAP4        BRKM5 
 0.081421086  0.006473257 -0.150000000  0.126904965  0.039863772  0.011568764 
       BRFS3       BPAC11        CRFB3        CCRO3        CMIG4        HGTX3 
 0.037796573 -0.010142985 -0.004665825 -0.009403152 -0.018038598  0.016595240 
       CIEL3        COGN3        CSAN3        CPFE3        CVCB3        CYRE3 
-0.001910725 -0.027501126  0.009529527  0.127049796 -0.009625067 -0.062617638 
       ECOR3        ELET3        ELET6        EMBR3        ENBR3        EGIE3 
 0.058404248 -0.146678752  0.150000000  0.127515548 -0.020202948  0.030370138 
       EQTL3        YDUQ3        FLRY3        GOAU4        GGBR4        GOLL4 
 0.074657313 -0.030839832  0.033632285  0.002721307 -0.079558633  0.013009613 
       HAPV3        HYPE3        IGTA3        GNDI3 

### Passo 2: Obter o IBOV para o Out-Sample

In [ ]:
ibov <- c(os.prices$IBOV)
ibov <- ibov/ibov[1]


# Função para aplicar o portifólio com rebalanceamento diário
apply_portfolio <- function(weights, prices, returns){
    result <- prices[names(weights)]
    rownames(result) <- 1:nrow(prices)
    result[1,] <- weights
    
    for(i in 1:nrow(returns)){
#       rebalanceamento sem alterar a estratégia
        next_day <- (1+returns[i,])*result[i,]
        money <- sum(next_day)
        result[i+1,] <- weights * money
        
#       Buy and Hold  
#         result[i+1,]<- (1+returns[i,])*result[i,]
    }
    return (rowSums(result));
}

### Obtenção do portifólio via otimização do CVaR

In [17]:
# iniciando o modelo de maximização
model = initialiseModel();
model$setModelFilename('cvar.lp')
model$setTimeLimit(3600)
model$setDirection(1)

cvar.scenarios <- nrow(insample.prices) - 1
cvar.ativos <- ncol(insample.prices)
cvar.alpha <- 0.05



model$addVariable("V", 1)

for (i in 1 : cvar.scenarios) model$addVariable(paste0("d", i), -1/(cvar.alpha*cvar.scenarios), 0, Inf)
for (i in 1 : cvar.ativos) model$addVariable(paste0("w", i), 0, -0.15, 0.15)

cvar.wi = paste0("w", seq(1: cvar.ativos))

for(i in 1 : cvar.scenarios){
    elements = paste0("w", seq(1, N));
    elements = c("V", paste0("d", i), cvar.wi)
    model$addConstraint(">=", 0, elements, values);
}

In [14]:
ncol(insample.prices)

[1] 71